In [2]:
import numpy as np

In [ ]:
# 25 tooth output
ratio1 = 2.1
ratio2 = 1.1
ratio_matrix = np.array([[ratio1, -1, 0, 0], [0, 0, ratio2, -1], [.5, .5, 0, 0], [0, 0, .5, .5]])

shaft_distance = 65 #mm
shaft_distance_matrix = np.array([0, 0, shaft_distance, shaft_distance])

gear_diameters = np.linalg.solve(ratio_matrix, shaft_distance_matrix)
# gear_diameters is input1, output1, input2, output2 in same units as shaft distance
print(gear_diameters)

[41.93548387 88.06451613 61.9047619  68.0952381 ]


In [18]:
# 21 tooth output
ratio1 = 1.7
ratio2 = .9
ratio_matrix = np.array([[ratio1, -1, 0, 0], [0, 0, ratio2, -1], [.5, .5, 0, 0], [0, 0, .5, .5]])

shaft_distance = 60 #mm 
shaft_distance_matrix = np.array([0, 0, shaft_distance, shaft_distance])

gear_diameters = np.linalg.solve(ratio_matrix, shaft_distance_matrix)
# gear_diameters is input1, output1, input2, output2 in same units as shaft distance
print(gear_diameters)

[44.44444444 75.55555556 63.15789474 56.84210526]


In [ ]:
# teeth calcs

#parameters
target_ratio_1 = 1.75
target_ratio_2 = .9

#constraints
Ni1 = 16
# all teeth numbers in order Ni1, N01, Ni2, No2, must be integers

#teeth numbers found with this equation
teeth_ratio_matrix = np.array([[1, 1, -1, -1], [target_ratio_1, -1, 0, 0], [0, 0, target_ratio_2, -1], [1, 0, 0, 0]])
b = np.array([0, 0, 0, Ni1])
teeth = np.linalg.solve(teeth_ratio_matrix, b)

def loss(ratio1, ratio2, target_ratio1, target_ratio2):
    return np.linalg.norm(ratio1-target_ratio1) + np.linalg.norm(ratio2-target_ratio2)


[16.         36.         23.11111111 28.88888889]


In [ ]:
#come back to a program like this but reformulate with a known axle to axle distance

# Target ratios
target_ratio_1 = 1.75
target_ratio_2 = 0.9

# Fixed value
Ni1 = 16

# Search range
search_range = range(10, 100)

best_loss = float('inf')
best_solution = None

for No1 in search_range:
    ratio1 = No1 / Ni1
    for Ni2 in search_range:
        for No2 in search_range:
            # Gear train constraint: Ni1 + No1 == Ni2 + No2
            if Ni1 + No1 != Ni2 + No2:
                continue

            ratio2 = No2 / Ni2
            loss = abs(ratio1 - target_ratio_1) + abs(ratio2 - target_ratio_2)
            if loss < best_loss:
                best_loss = loss
                best_solution = (Ni1, No1, Ni2, No2)

print(f"Best solution: Ni1={best_solution[0]}, No1={best_solution[1]}, Ni2={best_solution[2]}, No2={best_solution[3]}")
print(f"Ratios: {best_solution[1]/best_solution[0]:.4f}, {best_solution[3]/best_solution[2]:.4f}")
print(f"Loss: {best_loss:.6f}")


Best solution: Ni1=16, No1=28, Ni2=23, No2=21
Ratios: 1.7500, 0.9130
Loss: 0.013043
